## IBEA with DKL

In [1]:
from IBEAforBioemus.ibea import IBEA
import numpy as np 
from IBEAforBioemus.objectives_computation import evaluate_objectives_DKL, clear_files, compute_goal_distr, load_goal, saveIO
import warnings 
import time
import sys

''' 5 variables in this case are (in order): inh/exc ratio, max prob of connection, AMPA/NMDA ratio, NMDA syn weight, AMPA syn weight, synaptic noise.'''

warnings.filterwarnings("ignore",category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

clear_files("/home/ubuntu/bioemus/data", ".bin")
clear_files("/home/ubuntu/bioemus/config", ".json")
clear_files("/home/ubuntu/bioemus/config", ".txt")

# Save cell output in a custom class
original_stdout = sys.stdout
tee_stdout = saveIO(original_stdout)
sys.stdout = tee_stdout

alpha = 3
fun = evaluate_objectives_DKL
lbound = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.01])
# ubound = np.array([0.5, 0.5, 0.5, 1, 1, 0.1])
ubound = np.array([0.5, 1, 1, 1, 1, 0.1])

remaining_budget = np.inf
n_offspring = 1
max_generations = 1
var = (ubound-lbound)/15
print(f"lbound = {lbound}")
print(f"ubound = {ubound}")

goal = load_goal()
goal_distr = compute_goal_distr(goal)

solution = IBEA(kappa=0.05, alpha = alpha, n_offspring= n_offspring, max_generations= max_generations, seedit = 78, var=var, n_sbx=2, area_max=1)
start = time.time()
index, best_parameters, distances, true_distr, obj_monitor, fit_monitor = solution.ibea(fun, lbound, ubound, remaining_budget, goal_distr)
print(f"Best configuration is the number {index} with parameters {best_parameters} and distances {distances}")
end = time.time()
print(end-start)

# Save error values in a file
with open("monitor.txt", "w") as file:
   file.write(",".join(map(str, fit_monitor)))

# Save cell output in a .txt file
sys.stdout = original_stdout
captured_text = tee_stdout.getvalue()
with open("Output.txt", "w") as f:
    f.write(captured_text)

lbound = [0.1  0.1  0.1  0.1  0.1  0.01]
ubound = [0.5 1.  1.  1.  1.  0.1]
Starting optimization:
	Concluded emulation genesis0, parameters: [0.11927249 0.71286671 0.81882646 0.82009773 0.96876753 0.04996178]
	Concluded emulation genesis1, parameters: [0.43249395 0.32191087 0.14525009 0.5386886  0.68521398 0.09288809]
	Concluded emulation genesis2, parameters: [0.39549361 0.32267987 0.56622985 0.69767547 0.36351848 0.08861525]
Terminated parent generation, starting recombination


	Processing couple of childrens 1
	Concluded emulation genesis3, parameters: [0.12752621 0.79304777 0.84424826 0.82607053 0.93090206 0.04920266]
	Concluded emulation genesis4, parameters: [0.12316938 0.73379865 0.76344113 0.7342855  1.         0.0478089 ]
Terminated generation 1

OPTIMIZATION TERMINATED
Best configuration is the number 3 with parameters [0.12752621 0.79304777 0.84424826 0.82607053 0.93090206 0.04920266] and distances [0.37093115 0.0180081  0.12374749]
120.82951951026917


## IBEA with Medians

In [ ]:
from IBEAforBioemus.ibea import IBEA
import numpy as np 
from IBEAforBioemus.objectives_computation import evaluate_objectives_median, clear_files, compute_goal_median, load_goal
import scipy.io
import warnings 
import time
''' 5 variables in this case are (in order): inh/exc ratio, max prob of connection, AMPA/NMDA ratio, NMDA syn weight, AMPA syn weight, synaptic noise.'''

warnings.filterwarnings("ignore",category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

clear_files("/home/ubuntu/bioemus/data", ".bin")
clear_files("/home/ubuntu/bioemus/config", ".json")
clear_files("/home/ubuntu/bioemus/config", ".txt")

alpha = 200
fun = evaluate_objectives_median
lbound = np.array([0.1, 0.1, 0.1, 0.1, 0.05, 0.01])
ubound = np.array([1, 1, 1, 1, 1, 0.1])
remaining_budget = np.inf
n_offspring = 10
max_generations = 50
var = (ubound-lbound)/15
print(f"lbound = {lbound}")
print(f"ubound = {ubound}")


goal = load_goal()

goal_median = compute_goal_median(goal)

solution = IBEA(kappa=0.05, alpha = alpha, n_offspring= n_offspring, max_generations= max_generations, seedit = 32, var=var, n_sbx=2)
start = time.time()
index, best_parameters, distances, true_distr, obj_monitor, fit_monitor = solution.ibea(fun, lbound, ubound, remaining_budget, goal_median)
print(f"Best configuration is the number {index} with parameters {best_parameters} and distances {distances}")
end = time.time()
print(end-start)
with open("monitor.txt", "w") as file:
   file.write(",".join(map(str, fit_monitor)))

## Debug

In [ ]:
from IBEAforBioemus.objectives_computation import compute_objectives, compute_goal_distr, do_analysis, load_goal
import scipy
import numpy as np

goal = load_goal()

goal_distr = compute_goal_distr(goal)
CONFIG_NAME = 'genesis658'
RASTER_LIST = CONFIG_NAME
filename = "/home/ubuntu/bioemus/data/raster_" + CONFIG_NAME + ".bin"
individual_metrics = do_analysis(filename, RASTER_LIST, [0, 0, 0, 0, 0, 0])
distances, true_distributions = compute_objectives(individual_metrics, goal_distr)